# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [4]:
'''
Your Code
'''
import requests
from bs4 import BeautifulSoup
import time
import _thread
url = "https://www.ptt.cc/bbs/hotboards.html"

In [5]:
res = requests.get(url)
soup = BeautifulSoup(res.text)
lists = soup.find_all('div',{'class':'b-ent'})
startTime = time.time()
for list in lists:
    linkUrl = 'https://www.ptt.cc'.format(list.a['href'])
    res1 = requests.get(linkUrl)
finishTime = time.time()
print(finishTime - startTime)

66.05597686767578


In [6]:
res = requests.get(url)
soup = BeautifulSoup(res.text)
lists = soup.find_all('div',{'class':'b-ent'})
startTime = time.time()
for list in lists:
    linkUrl = 'https://www.ptt.cc'.format(list.a['href'])
    _thread.start_new_thread( requests.get, (linkUrl, ) )
finishTime = time.time()
print(finishTime - startTime)

0.02800154685974121


In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import _thread

In [2]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
#     for p in news_content:
#         """
#         .string屬性說明：
#         (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
#         (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
#         (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
#         """
#         if ((p.string) is not None):
#             print(p.string)

In [4]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

browser = webdriver.Chrome(executable_path='chromedriver')
# browser = webdriver.Chrome(executable_path='./Data/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight;")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    current_height = browser.execute_script("return document.body.scrollHeight;")
    
    if current_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = current_height
        continue
        
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})
news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

# 關閉瀏覽器
browser.quit();

網頁更新中...
到達頁面底端


In [5]:
# 單線程
startTime = time.time()
for i, news_item in enumerate(news_block):
#     print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
#     print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)
    finishTime = time.time()
print(finishTime - startTime)

9.145347118377686


In [6]:
# 多線程
startTime = time.time()
for i, news_item in enumerate(news_block):
#     print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
#     print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    _thread.start_new_thread(getNewsDetailContent, (externalLink, ))
    finishTime = time.time()
print(finishTime - startTime)

0.013321638107299805


In [9]:
from bs4 import BeautifulSoup
import requests
import re
import time
import threading
import json
from urllib.parse import urljoin
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

In [10]:
#單線程爬蟲
def crawl_article(url):
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-content')
    
    # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline') #list
    author = ''
    title = ''
    date = ''
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
    
    # 取得留言區主體
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
    
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
    
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
    
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    # 處理留言區
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        # 過濾額外空白與換行符號
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

        # 整理打包留言的資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
    
    # 統計推噓文
    # count 為推噓文相抵看這篇文章推文還是噓文比較多
    # all 為總共留言數量 
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

import time

# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []

stime = time.time()
# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        if a_title:
            article_URL = urljoin(PTT_URL, a_title['href'])
        else:
            article_URL = None
            a_title = '<a>本文已刪除</a>'
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 呼叫上面寫好的 function 來對文章進行爬蟲
        if article_URL:
            parse_data = crawl_article(article_URL) # 返回單一文章資訊的字典
        
        # 將爬完的資料儲存
        all_data.append(parse_data)
        
etime = time.time()
print('共用時：',etime-stime )

Parse [問卦] 可以痛扁還去日本玩的人嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1582875823.A.33D.html
Parse Re: [新聞] 洪慈庸將任立院顧問 游錫堃：機要缺不算 - https://www.ptt.cc/bbs/Gossiping/M.1582875848.A.602.html
Parse [問卦] 德義日韓算是新一代舔共軸心國嗎? - https://www.ptt.cc/bbs/Gossiping/M.1582875901.A.2AE.html
Parse Re: [問卦] 對面套房內有基地台怎麼辦？ - https://www.ptt.cc/bbs/Gossiping/M.1582875992.A.3A3.html
Parse [新聞] 當年和平醫院逾一成陪病者染SARS 名醫籲 - https://www.ptt.cc/bbs/Gossiping/M.1582875995.A.5F5.html
Parse [問卦] 求神拜佛有用的話 是不是醫院都可以關了 - https://www.ptt.cc/bbs/Gossiping/M.1582876006.A.96A.html
Parse [問卦] 只有我會希望公司有人生病嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1582876025.A.9EB.html
Parse [問卦] 車禍幫忙抬車,如果手滑,車子掉下壓死人? - https://www.ptt.cc/bbs/Gossiping/M.1582876042.A.759.html
Parse [問卦] 去日本前要儲值台灣價值嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1582876115.A.045.html
Parse [問卦] 顧門口的員工薪水多少比較適合??? - https://www.ptt.cc/bbs/Gossiping/M.1582876117.A.D94.html
Parse [問卦] 開蟑聖母現在在想什麼？ - https://www.ptt.cc/bbs/Gossiping/M.1582876145.A.0CA.html
Parse Re: [新聞] 富二代率眾毆死保全 公訴不受理 - ht

In [11]:
#多線程爬蟲
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

def crawl_article(url):
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-content')
    
    # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline') #list
    author = ''
    title = ''
    date = ''
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
    
    # 取得留言區主體
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
    
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
    
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
    
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    # 處理留言區
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        # 過濾額外空白與換行符號
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

        # 整理打包留言的資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
    
    # 統計推噓文
    # count 為推噓文相抵看這篇文章推文還是噓文比較多
    # all 為總共留言數量 
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

import time

# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []
all_url = []

stime = time.time()
# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        if a_title:
            article_URL = urljoin(PTT_URL, a_title['href'])
        else:
            article_URL = None
            a_title = '<a>本文已刪除</a>'
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 把文章連結存在list
        if article_URL:
            all_url.append(article_URL)

# 從這裡丟給子執行緒工作            
# 建立 n 個子執行緒，分別去抓文章內容
threads = []
for i in range(len(all_url)):
    threads.append(threading.Thread(target = crawl_article, args = (all_url[i],)))
    threads[i].start()

# 主執行緒繼續執行自己的工作
# ...

# 等待所有子執行緒結束
for i in range(len(all_url)):
    threads[i].join()

        
etime = time.time()
print('共用時：',etime-stime )

Parse [問卦] 可以痛扁還去日本玩的人嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1582875823.A.33D.html
Parse Re: [新聞] 洪慈庸將任立院顧問 游錫堃：機要缺不算 - https://www.ptt.cc/bbs/Gossiping/M.1582875848.A.602.html
Parse [問卦] 德義日韓算是新一代舔共軸心國嗎? - https://www.ptt.cc/bbs/Gossiping/M.1582875901.A.2AE.html
Parse Re: [問卦] 對面套房內有基地台怎麼辦？ - https://www.ptt.cc/bbs/Gossiping/M.1582875992.A.3A3.html
Parse [新聞] 當年和平醫院逾一成陪病者染SARS 名醫籲 - https://www.ptt.cc/bbs/Gossiping/M.1582875995.A.5F5.html
Parse [問卦] 求神拜佛有用的話 是不是醫院都可以關了 - https://www.ptt.cc/bbs/Gossiping/M.1582876006.A.96A.html
Parse [問卦] 只有我會希望公司有人生病嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1582876025.A.9EB.html
Parse [問卦] 車禍幫忙抬車,如果手滑,車子掉下壓死人? - https://www.ptt.cc/bbs/Gossiping/M.1582876042.A.759.html
Parse [問卦] 去日本前要儲值台灣價值嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1582876115.A.045.html
Parse [問卦] 顧門口的員工薪水多少比較適合??? - https://www.ptt.cc/bbs/Gossiping/M.1582876117.A.D94.html
Parse [問卦] 開蟑聖母現在在想什麼？ - https://www.ptt.cc/bbs/Gossiping/M.1582876145.A.0CA.html
Parse Re: [新聞] 富二代率眾毆死保全 公訴不受理 - ht

In [12]:
#物件導向寫法
class MyTask(threading.Thread):
    def __init__(self, task_name):
        super(MyTask, self).__init__()
        self.task_name = task_name

    def run(self):
        print("Get task: {}\n".format(self.task_name))
        time.sleep(1)
        print("Finish task: {}\n".format(self.task_name))



if __name__ == "__main__":
    data = [1,2,3,4,5,6,7,8,9,10]
    tasks = []
    for i in range(0, 10):
        # 建立 task
        tasks.append(MyTask("task_{}".format(data[i])))
    for t in tasks:
        # 開始執行 task
        t.start()

    for t in tasks:
        # 等待 task 執行完畢
        # 完畢前會阻塞住主執行緒
        t.join()
    print("Finish.")

Get task: task_1

Get task: task_2

Get task: task_3

Get task: task_4

Get task: task_5

Get task: task_6

Get task: task_7

Get task: task_8

Get task: task_9

Get task: task_10

Finish task: task_1

Finish task: task_2

Finish task: task_3

Finish task: task_4

Finish task: task_5

Finish task: task_6

Finish task: task_7

Finish task: task_8

Finish task: task_9

Finish task: task_10

Finish.


In [13]:
#改寫
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

class Crawl_Article(threading.Thread):
    
    def __init__(self, url):
        super(Crawl_Article, self).__init__()
        self.url = url

    # 原crawl_article，改成子執行緒run任務
    def run(self): 
        print("Get子執行緒: {}\n".format(self.url))

        response = requests.get(self.url, cookies={'over18': '1'})

        # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
        if response.status_code != 200:
            print('Error - {} is not available to access'.format(self.url))
            return

        # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
        soup = BeautifulSoup(response.text)

        # 取得文章內容主體
        main_content = soup.find(id='main-content')

        # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
        metas = main_content.select('div.article-metaline') #list
        author = ''
        title = ''
        date = ''
        if metas:
            if metas[0].select('span.article-meta-value')[0]:
                author = metas[0].select('span.article-meta-value')[0].string
            if metas[1].select('span.article-meta-value')[0]:
                title = metas[1].select('span.article-meta-value')[0].string
            if metas[2].select('span.article-meta-value')[0]:
                date = metas[2].select('span.article-meta-value')[0].string

            # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
            #
            # .extract() 方法可以參考官方文件
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
            for m in metas:
                m.extract()
            for m in main_content.select('div.article-metaline-right'):
                m.extract()

        # 取得留言區主體
        pushes = main_content.find_all('div', class_='push')
        for p in pushes:
            p.extract()

        # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
        # 透過 regular expression 取得 IP
        # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
        try:
            ip = main_content.find(text=re.compile(u'※ 發信站:'))
            ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
        except Exception as e:
            ip = ''

        # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
        # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
        #
        # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
        filtered = []
        for v in main_content.stripped_strings:
            # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
            if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                filtered.append(v)

        # 定義一些特殊符號與全形符號的過濾器
        expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
        filtered = [i for i in filtered if i]
        content = ' '.join(filtered)

        # 處理留言區
        # p 計算推文數量
        # b 計算噓文數量
        # n 計算箭頭數量
        p, b, n = 0, 0, 0
        messages = []
        for push in pushes:
            # 假如留言段落沒有 push-tag 就跳過
            if not push.find('span', 'push-tag'):
                continue

            # 過濾額外空白與換行符號
            # push_tag 判斷是推文, 箭頭還是噓文
            # push_userid 判斷留言的人是誰
            # push_content 判斷留言內容
            # push_ipdatetime 判斷留言日期時間
            push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
            push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
            push_content = push.find('span', 'push-content').strings
            push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
            push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

            # 整理打包留言的資訊, 並統計推噓文數量
            messages.append({
                'push_tag': push_tag,
                'push_userid': push_userid,
                'push_content': push_content,
                'push_ipdatetime': push_ipdatetime})
            if push_tag == u'推':
                p += 1
            elif push_tag == u'噓':
                b += 1
            else:
                n += 1

        # 統計推噓文
        # count 為推噓文相抵看這篇文章推文還是噓文比較多
        # all 為總共留言數量 
        message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

        # 整理文章資訊
        data = {
            'url': self.url,
            'article_author': author,
            'article_title': title,
            'article_date': date,
            'article_content': content,
            'ip': ip,
            'message_count': message_count,
            'messages': messages
        }
        return data

import time

if __name__ == '__main__':
    
    # 對文章列表送出請求並取得列表主體
    resp = requests.get(PTT_URL, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text)
    main_list = soup.find('div', class_='bbs-screen')
    all_data = []
    all_url = []

    stime = time.time()
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break
        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            if a_title:
                article_URL = urljoin(PTT_URL, a_title['href'])
            else:
                article_URL = None
                a_title = '<a>本文已刪除</a>'
            article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
            # 把文章連結存在list
            if article_URL:
                all_url.append(article_URL)
    
    print('共{}個連結'.format(len(all_url)))
    # 從這裡丟給子執行緒工作            
    # 建立 n 個子執行緒，分別去抓文章內容
    threads = []
    for i in range(len(all_url)):
        threads.append(Crawl_Article(all_url[i]))
        threads[i].start()

    # 主執行緒繼續執行自己的工作
    # ...

    # 等待所有子執行緒結束
    for i in range(len(all_url)):
        threads[i].join()


    etime = time.time()
    print('共用時：',etime-stime )

Parse Re: [新聞] 富二代率眾毆死保全 公訴不受理 - https://www.ptt.cc/bbs/Gossiping/M.1582876536.A.B2E.html
Parse Re: [新聞]拍電影燒光3千萬 阿Ken亮出6字底牌 - https://www.ptt.cc/bbs/Gossiping/M.1582876557.A.92D.html
Parse Re: [問卦] 開蟑聖母現在在想什麼？ - https://www.ptt.cc/bbs/Gossiping/M.1582876570.A.5F5.html
Parse Re: [爆卦] 第三十三案、第三十四案 LIVE記者會 - https://www.ptt.cc/bbs/Gossiping/M.1582876582.A.45D.html
Parse [問卦] 欸！究竟怎麼防中國網軍啊？ - https://www.ptt.cc/bbs/Gossiping/M.1582876639.A.08A.html
Reach the last article
共5個連結
Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876536.A.B2E.html

Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876557.A.92D.html
Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876570.A.5F5.html


Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876582.A.45D.html

Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876639.A.08A.html

共用時： 0.5022680759429932


In [14]:
# 使用佇列 Queue
from queue import Queue

PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

class Crawl_Article(threading.Thread):
    
    def __init__(self, queue):
        super(Crawl_Article, self).__init__()
        self.queue = queue

    # 原crawl_article，改成子執行緒run任務
    def run(self): 
        # 當 queue 裡面有資料再執行
        while self.queue.qsize() > 0:
            url = self.queue.get()
            print("Get子執行緒: {}\n".format(url))

            response = requests.get(url, cookies={'over18': '1'})

            # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
            if response.status_code != 200:
                print('Error - {} is not available to access'.format(url))
                return

            # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
            soup = BeautifulSoup(response.text)

            # 取得文章內容主體
            main_content = soup.find(id='main-content')

            # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
            metas = main_content.select('div.article-metaline') #list
            author = ''
            title = ''
            date = ''
            if metas:
                if metas[0].select('span.article-meta-value')[0]:
                    author = metas[0].select('span.article-meta-value')[0].string
                if metas[1].select('span.article-meta-value')[0]:
                    title = metas[1].select('span.article-meta-value')[0].string
                if metas[2].select('span.article-meta-value')[0]:
                    date = metas[2].select('span.article-meta-value')[0].string

                # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
                #
                # .extract() 方法可以參考官方文件
                #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
                for m in metas:
                    m.extract()
                for m in main_content.select('div.article-metaline-right'):
                    m.extract()

            # 取得留言區主體
            pushes = main_content.find_all('div', class_='push')
            for p in pushes:
                p.extract()

            # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
            # 透過 regular expression 取得 IP
            # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
            try:
                ip = main_content.find(text=re.compile(u'※ 發信站:'))
                ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
            except Exception as e:
                ip = ''

            # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
            # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
            #
            # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
            filtered = []
            for v in main_content.stripped_strings:
                # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
                if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                    filtered.append(v)

            # 定義一些特殊符號與全形符號的過濾器
            expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
            filtered = [i for i in filtered if i]
            content = ' '.join(filtered)

            # 處理留言區
            # p 計算推文數量
            # b 計算噓文數量
            # n 計算箭頭數量
            p, b, n = 0, 0, 0
            messages = []
            for push in pushes:
                # 假如留言段落沒有 push-tag 就跳過
                if not push.find('span', 'push-tag'):
                    continue

                # 過濾額外空白與換行符號
                # push_tag 判斷是推文, 箭頭還是噓文
                # push_userid 判斷留言的人是誰
                # push_content 判斷留言內容
                # push_ipdatetime 判斷留言日期時間
                push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
                push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
                push_content = push.find('span', 'push-content').strings
                push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
                push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

                # 整理打包留言的資訊, 並統計推噓文數量
                messages.append({
                    'push_tag': push_tag,
                    'push_userid': push_userid,
                    'push_content': push_content,
                    'push_ipdatetime': push_ipdatetime})
                if push_tag == u'推':
                    p += 1
                elif push_tag == u'噓':
                    b += 1
                else:
                    n += 1

            # 統計推噓文
            # count 為推噓文相抵看這篇文章推文還是噓文比較多
            # all 為總共留言數量 
            message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

            # 整理文章資訊
            data = {
                'url': url,
                'article_author': author,
                'article_title': title,
                'article_date': date,
                'article_content': content,
                'ip': ip,
                'message_count': message_count,
                'messages': messages
            }
            return data

import time

if __name__ == '__main__':
    
    # 對文章列表送出請求並取得列表主體
    resp = requests.get(PTT_URL, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text)
    main_list = soup.find('div', class_='bbs-screen')
    all_data = []
    
    Q_url = Queue()

    stime = time.time()
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break
        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            if a_title:
                article_URL = urljoin(PTT_URL, a_title['href'])
            else:
                article_URL = None
                a_title = '<a>本文已刪除</a>'
            article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
            # 把文章連結存在list
            if article_URL:
                Q_url.put(article_URL)
    
    print('共{}個連結'.format(Q_url.qsize()))
    
    # 從這裡丟給子執行緒工作            
    # 建立 n 個子執行緒，分別去抓文章內容
    threads = []
    for i in range(Q_url.qsize()):
        threads.append(Crawl_Article(Q_url))
        threads[i].start()

    # 主執行緒繼續執行自己的工作
    # ...

    # 等待所有子執行緒結束
    for i in range(len(all_url)):
        threads[i].join()


    etime = time.time()
    print('共用時：',etime-stime )
    

Parse Re: [新聞] 富二代率眾毆死保全 公訴不受理 - https://www.ptt.cc/bbs/Gossiping/M.1582876536.A.B2E.html
Parse Re: [新聞]拍電影燒光3千萬 阿Ken亮出6字底牌 - https://www.ptt.cc/bbs/Gossiping/M.1582876557.A.92D.html
Parse Re: [問卦] 開蟑聖母現在在想什麼？ - https://www.ptt.cc/bbs/Gossiping/M.1582876570.A.5F5.html
Parse Re: [爆卦] 第三十三案、第三十四案 LIVE記者會 - https://www.ptt.cc/bbs/Gossiping/M.1582876582.A.45D.html
Parse [問卦] 欸！究竟怎麼防中國網軍啊？ - https://www.ptt.cc/bbs/Gossiping/M.1582876639.A.08A.html
Parse Re: [新聞] 武漢肺炎》居家檢疫失聯 新竹縣府協尋「 - https://www.ptt.cc/bbs/Gossiping/M.1582876679.A.EBA.html
Parse [問卦] 這次日本會不會蓋牌得太誇張 - https://www.ptt.cc/bbs/Gossiping/M.1582876682.A.C63.html
Parse Re: [新聞]拍電影燒光3千萬 阿Ken亮出6字底牌 - https://www.ptt.cc/bbs/Gossiping/M.1582876702.A.579.html
Reach the last article
共8個連結
Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876536.A.B2E.html

Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876557.A.92D.html

Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1582876570.A.5F5.html

Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.15828765

In [15]:
# 使用lock:
# 被 Lock 的 acquire 與 release 包起來的這段程式碼不會被兩個執行緒同時執行。 用來寫入檔案
class Worker(threading.Thread):
    
    def __init__(self, queue, num, lock):
        
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num
        self.lock = lock

    def run(self):
        while self.queue.qsize() > 0:
            url = self.queue.get()

            # 取得 lock
            lock.acquire()
            print("子執行緒 %d 取得lock" % self.num)

            # 不能讓多個執行緒同時進的工作
            print("子執行緒 %d: 寫入檔案 %s" % (self.num, url))
            time.sleep(1)

            # 釋放 lock
            print("子執行緒 %d 釋放lock" % self.num)
            self.lock.release()
            
#建立一個佇列
my_queue = Queue()

#假裝放五個URL進去queue
for i in range(1,6):
    my_queue.put("Url %d" % i)

# 建立 lock
lock = threading.Lock()

#建立2個子執行緒，傳入queue和一個參數和lock
my_worker1 = Worker(my_queue, 100, lock)
my_worker2 = Worker(my_queue, 200, lock)

my_worker1.start()
my_worker2.start()

my_worker1.join()
my_worker2.join()

print("Done.")

子執行緒 100 取得lock
子執行緒 100: 寫入檔案 Url 1
子執行緒 100 釋放lock
子執行緒 200 取得lock
子執行緒 200: 寫入檔案 Url 2
子執行緒 200 釋放lock
子執行緒 200 取得lock
子執行緒 200: 寫入檔案 Url 4
子執行緒 200 釋放lock
子執行緒 100 取得lock
子執行緒 100: 寫入檔案 Url 3
子執行緒 100 釋放lock
子執行緒 200 取得lock
子執行緒 200: 寫入檔案 Url 5
子執行緒 200 釋放lock
Done.


In [ ]:
# 改寫：加入能寫入檔案的lock __有問題
from queue import Queue

PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

class Crawl_Article(threading.Thread):
    
    def __init__(self, queue, lock):
        super(Crawl_Article, self).__init__()
        self.queue = queue
        self.lock = lock

    # 原crawl_article，改成子執行緒run任務
    def run(self): 
        # 當 queue 裡面有資料再執行
        while self.queue.qsize() > 0:
            url = self.queue.get()
            #print("Get子執行緒: {}\n".format(url))

            response = requests.get(url, cookies={'over18': '1'})

            # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
            if response.status_code != 200:
                print('Error - {} is not available to access'.format(url))
                return

            # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
            soup = BeautifulSoup(response.text)

            # 取得文章內容主體
            main_content = soup.find(id='main-content')

            # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
            metas = main_content.select('div.article-metaline') #list
            author = ''
            title = ''
            date = ''
            if metas:
                if metas[0].select('span.article-meta-value')[0]:
                    author = metas[0].select('span.article-meta-value')[0].string
                if metas[1].select('span.article-meta-value')[0]:
                    title = metas[1].select('span.article-meta-value')[0].string
                if metas[2].select('span.article-meta-value')[0]:
                    date = metas[2].select('span.article-meta-value')[0].string

                # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
                #
                # .extract() 方法可以參考官方文件
                #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
                for m in metas:
                    m.extract()
                for m in main_content.select('div.article-metaline-right'):
                    m.extract()

            # 取得留言區主體
            pushes = main_content.find_all('div', class_='push')
            for p in pushes:
                p.extract()

            # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
            # 透過 regular expression 取得 IP
            # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
            try:
                ip = main_content.find(text=re.compile(u'※ 發信站:'))
                ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
            except Exception as e:
                ip = ''

            # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
            # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
            #
            # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
            filtered = []
            for v in main_content.stripped_strings:
                # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
                if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                    filtered.append(v)

            # 定義一些特殊符號與全形符號的過濾器
            expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
            filtered = [i for i in filtered if i]
            content = ' '.join(filtered)

            # 處理留言區
            # p 計算推文數量
            # b 計算噓文數量
            # n 計算箭頭數量
            p, b, n = 0, 0, 0
            messages = []
            for push in pushes:
                # 假如留言段落沒有 push-tag 就跳過
                if not push.find('span', 'push-tag'):
                    continue

                # 過濾額外空白與換行符號
                # push_tag 判斷是推文, 箭頭還是噓文
                # push_userid 判斷留言的人是誰
                # push_content 判斷留言內容
                # push_ipdatetime 判斷留言日期時間
                push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
                push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
                push_content = push.find('span', 'push-content').strings
                push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
                push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

                # 整理打包留言的資訊, 並統計推噓文數量
                messages.append({
                    'push_tag': push_tag,
                    'push_userid': push_userid,
                    'push_content': push_content,
                    'push_ipdatetime': push_ipdatetime})
                if push_tag == u'推':
                    p += 1
                elif push_tag == u'噓':
                    b += 1
                else:
                    n += 1

            # 統計推噓文
            # count 為推噓文相抵看這篇文章推文還是噓文比較多
            # all 為總共留言數量 
            message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

            # 整理文章資訊
            data = {
                'url': url,
                'article_author': author,
                'article_title': title,
                'article_date': date,
                'article_content': content,
                'ip': ip,
                'message_count': message_count,
                'messages': messages
            }
            
            
            # 寫入檔案:單一文章內容
            
            # 取得 lock
            lock.acquire()
            #print("%s 取得lock" % url[32:51])

            # 不能讓多個執行緒同時進的工作 : 將爬完的資訊存成 json 檔案
            #print("寫入檔案")
            with open('../Data/PTT_Article.json', 'a+', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
                f.write(",")

            # 釋放 lock
            #print("%s 釋放lock" % url[32:51])
            self.lock.release()

import time

if __name__ == '__main__':
    
    # 對文章列表送出請求並取得列表主體
    resp = requests.get(PTT_URL, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text)
    main_list = soup.find('div', class_='bbs-screen')
    all_data = []
    
    Q_url = Queue()

    stime = time.time()
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break
        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            
            if a_title:
                article_URL = urljoin(PTT_URL, a_title['href'])
                article_title = a_title.text
            else:
                article_URL = None
                a_title = '<a>本文已刪除</a>'
                article_title = a_title
                
            #article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
            # 把文章連結存在list
            if article_URL:
                Q_url.put(article_URL)
    
    print('共{}個連結'.format(Q_url.qsize()))
    
    # 建立 lock
    lock = threading.Lock()
    
    # 從這裡丟給子執行緒工作            
    # 建立 n 個子執行緒，分別去抓文章內容
    threads = []
    for i in range(Q_url.qsize()):
        threads.append(Crawl_Article(Q_url, lock))
        threads[i].start()

    # 主執行緒繼續執行自己的工作
    # ...

    # 等待所有子執行緒結束
    for i in range(len(all_url)):
        threads[i].join()


    etime = time.time()
    print('共用時：',etime-stime )

Parse Re: [新聞] 富二代率眾毆死保全 公訴不受理 - https://www.ptt.cc/bbs/Gossiping/M.1582876536.A.B2E.html
Parse Re: [新聞]拍電影燒光3千萬 阿Ken亮出6字底牌 - https://www.ptt.cc/bbs/Gossiping/M.1582876557.A.92D.html
Parse Re: [問卦] 開蟑聖母現在在想什麼？ - https://www.ptt.cc/bbs/Gossiping/M.1582876570.A.5F5.html
Parse Re: [爆卦] 第三十三案、第三十四案 LIVE記者會 - https://www.ptt.cc/bbs/Gossiping/M.1582876582.A.45D.html
Parse [問卦] 欸！究竟怎麼防中國網軍啊？ - https://www.ptt.cc/bbs/Gossiping/M.1582876639.A.08A.html
Parse Re: [新聞] 武漢肺炎》居家檢疫失聯 新竹縣府協尋「 - https://www.ptt.cc/bbs/Gossiping/M.1582876679.A.EBA.html
Parse [問卦] 這次日本會不會蓋牌得太誇張 - https://www.ptt.cc/bbs/Gossiping/M.1582876682.A.C63.html
Parse Re: [新聞]拍電影燒光3千萬 阿Ken亮出6字底牌 - https://www.ptt.cc/bbs/Gossiping/M.1582876702.A.579.html
Parse Re: [問卦] 人類有沒有可能只吃一種食物就能活 - https://www.ptt.cc/bbs/Gossiping/M.1582876714.A.0B6.html
Reach the last article
共9個連結


Exception in thread Thread-53:
Traceback (most recent call last):
  File "C:\Users\au\Anaconda3\envs\spider\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-16-aecb56849ec5>", line 150, in run
    with open('../Data/PTT_Article.json', 'a+', encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: '../Data/PTT_Article.json'



In [7]:
import os
import requests
import re
import time
import threading
from bs4 import BeautifulSoup

In [8]:
class myThread (threading.Thread):
    def __init__(self, key_word_link, key_word, recursive):
        threading.Thread.__init__(self)
        self.key_word_link = key_word_link
        self.key_word = key_word
        self.recursive = recursive
    def run(self):
        WikiArticle(self.key_word_link, self.key_word, self.recursive, multithread=True)

def WikiArticle(key_word_link, key_word, recursive, multithread=False):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        '''
        
        Your code here
        
        '''
        dir_ = 'WikiArticle' #存放資料夾
        if not os.path.exists(dir_):
            os.makedirs(dir_)
        
        with open('{}/{}.txt'.format(dir_, key_word.replace('/', '_')), 'w', encoding='utf8') as f_:
            for paragraph in content:
                f_.write(paragraph.get_text())
            

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        '''
        
        Your code here
        
        '''
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_link] = a_keyword

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            thread_list = []
            for k, v in external_link_dict.items():
                if multithread:
                    thread = myThread(k, v, recursive)
                    thread_list.append(thread)
                    thread.start()
                else:
                    WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                    
            if multithread:
                for thread in thread_list:
                    thread.join()

In [11]:
input_keyword = "RNN"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

RNN: RNN
/wiki/RNN


In [13]:
# 單線程爬蟲
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

startTime = time.time()
WikiArticle(root_keyword_link, input_keyword, 0)
finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間


遞迴層[0] - /wiki/RNN (RNN)
遞迴層[1] - /wiki/%E6%A2%AF%E5%BA%A6%E6%B6%88%E5%A4%B1%E9%97%AE%E9%A2%98 (梯度消失問題)
遞迴層[1] - /wiki/LSTM (LSTM)
遞迴層[1] - /wiki/%E5%89%8D%E9%A6%88%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C (前饋神經網絡)
遞迴層[1] - /wiki/%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97 (時間序列)
遞迴層[1] - /wiki/%E6%89%8B%E5%86%99%E8%AF%86%E5%88%AB (手寫識別)
遞迴層[1] - /wiki/Hopfield%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C (Hopfield神經網絡)
遞迴層[1] - /wiki/%E7%99%BE%E5%BA%A6 (百度)
遞迴層[1] - /wiki/%E8%AF%AD%E9%9F%B3%E5%90%88%E6%88%90 (文本到語音合成)
遞迴層[1] - /wiki/Android (安卓系統)
遞迴層[1] - /wiki/%E6%9C%BA%E5%99%A8%E7%BF%BB%E8%AF%91 (機器翻譯)
遞迴層[1] - /wiki/%E8%AA%9E%E8%A8%80%E6%A8%A1%E5%9E%8B (語言建模)
遞迴層[1] - /wiki/%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C (卷積神經網絡)
遞迴層[1] - /wiki/%E5%9B%BE%E5%83%8F%E8%87%AA%E5%8A%A8%E6%A0%87%E6%B3%A8 (圖像自動標註)
遞迴層[1] - /wiki/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86 (自然語言處理)
遞迴層[1] - /wiki/%E9%9D%9E%E7%BA%BF%E6%80%A7 (非線性方程)
遞迴層[1] - /wiki/%E9%80%BB%E8%BE%91%E5%87%BD%E6%95%B0 (Logisti

In [14]:
# 多線程爬蟲
startTime = time.time()
WikiArticle(root_keyword_link, input_keyword, 0, multithread=True)
finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

遞迴層[0] - /wiki/RNN (RNN)
遞迴層[1] - /wiki/%E6%A2%AF%E5%BA%A6%E6%B6%88%E5%A4%B1%E9%97%AE%E9%A2%98 (梯度消失問題)遞迴層[1] - /wiki/LSTM (LSTM)

遞迴層[1] - /wiki/%E5%89%8D%E9%A6%88%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C (前饋神經網絡)
遞迴層[1] - /wiki/%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97 (時間序列)
遞迴層[1] - /wiki/%E6%89%8B%E5%86%99%E8%AF%86%E5%88%AB (手寫識別)
遞迴層[1] - /wiki/Hopfield%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C (Hopfield神經網絡)
遞迴層[1] - /wiki/%E7%99%BE%E5%BA%A6 (百度)
遞迴層[1] - /wiki/%E8%AF%AD%E9%9F%B3%E5%90%88%E6%88%90 (文本到語音合成)
遞迴層[1] - /wiki/Android (安卓系統)
遞迴層[1] - /wiki/%E6%9C%BA%E5%99%A8%E7%BF%BB%E8%AF%91 (機器翻譯)
遞迴層[1] - /wiki/%E8%AA%9E%E8%A8%80%E6%A8%A1%E5%9E%8B (語言建模)
遞迴層[1] - /wiki/%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C (卷積神經網絡)
遞迴層[1] - /wiki/%E5%9B%BE%E5%83%8F%E8%87%AA%E5%8A%A8%E6%A0%87%E6%B3%A8 (圖像自動標註)遞迴層[1] - /wiki/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86 (自然語言處理)

遞迴層[1] - /wiki/%E9%9D%9E%E7%BA%BF%E6%80%A7 (非線性方程)
遞迴層[1] - /wiki/%E9%80%BB%E8%BE%91%E5%87%BD%E6%95%B0 (Logisti

In [15]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def get_SO2(site):
    browser = webdriver.Chrome(executable_path='chromedriver')

    browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

    # 模擬使用者操作行為，選擇/點擊

    selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
    selectSite.select_by_value(site)
    selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
    selectYear.select_by_value('2018')

    browser.find_element_by_id('ctl05_btnQuery').click()

    locator=("id", "ctl05_gv")
    try:
        WebDriverWait(browser, 30, 0.5).until(EC.presence_of_element_located(locator))
    finally:
        print('')

    # 取得資料，丟到 BeautifulSoup 解析

    html_source = browser.page_source

    soup = BeautifulSoup(html_source, 'html.parser')
    table = soup.find('table', class_='TABLE_G')
    trs=table.find_all('tr')

    bool_so2=False
    str_date=''
    str_value=''
    for tr in trs:
        tds=tr.find_all('td')
        if (len(tds)>0):
            if (len(tds)==5):
                bool_so2=tds[0].text=='SO2'
                if (bool_so2):
                    str_date=tds[2].text
                    str_value=tds[3].text
            else:
                str_date=tds[0].text
                str_value=tds[1].text
        if (bool_so2):
            if (str_date<='2018/08'):
                print(f"{str_date} / {str_value}")

In [16]:
import time

startTime = time.time()
get_SO2('11')
get_SO2('6')
finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間


2018/01 / 1.80
2018/02 / 1.90
2018/03 / 2.20
2018/04 / 2.30
2018/05 / 3.10
2018/06 / 2.70
2018/07 / 2.20
2018/08 / 2.40

2018/01 / 2.40
2018/02 / 2.60
2018/03 / 2.90
2018/04 / 3.20
2018/05 / 4.20
2018/06 / 3
2018/07 / 3.10
2018/08 / 3.90
47.20784139633179


In [17]:
import _thread

sites = ['11', '6']

startTime = time.time()

for site in sites:
    _thread.start_new_thread( get_SO2, (site,) )

finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

0.0


2018/01 / 1.80
2018/01 / 2.402018/02 / 1.90
2018/03 / 2.20
2018/04 / 2.30
2018/05 / 3.10
2018/06 / 2.70
2018/07 / 2.20
2018/08 / 2.40

2018/02 / 2.60
2018/03 / 2.90
2018/04 / 3.20
2018/05 / 4.20
2018/06 / 3
2018/07 / 3.10
2018/08 / 3.90
